In [1]:
from typing import Any, Mapping, Sequence


def access_nested_map(nested_map: Mapping, path: Sequence) -> Any:
    """Access nested map with key path.
    Parameters
    ----------
    nested_map: Mapping
        A nested map
    path: Sequence
        a sequence of key representing a path to the value
    Example
    -------
    >>> nested_map = {"a": {"b": {"c": 1}}}
    >>> access_nested_map(nested_map, ["a", "b", "c"])
    1
    """
    for key in path:
        if not isinstance(nested_map, Mapping):
            raise KeyError(key)
        nested_map = nested_map[key]

    return nested_map

In [6]:
nested_map = {"a": {"b": {"c": {"d": 'Emmanuel'}}}}
str(nested_map)
# access_nested_map(nested_map, ["a", "b", "c","d"])

"{'a': {'b': {'c': {'d': 'Emmanuel'}}}}"

In [4]:
from typing import Dict
import requests
def get_json(url: str) -> Dict:
    """Get JSON from remote URL.
    """
    response = requests.get(url)
    return response.json()

In [3]:
import datetime
from unittest.mock import Mock

# Save a couple of test days
tuesday = datetime.datetime(year=2019, month=1, day=1)
saturday = datetime.datetime(year=2019, month=1, day=5)

# Mock datetime to control today's date
datetime = Mock()

def is_weekday():
    today = datetime.datetime.today()
    # Python's datetime library treats Monday as 0 and Sunday as 6
    # print(today.weekday())
    return (0 <= today.weekday() < 5)

# Mock .today() to return Tuesday
datetime.datetime.today.return_value = tuesday
# Test Tuesday is a weekday
assert is_weekday()
# Mock .today() to return Saturday
datetime.datetime.today.return_value = saturday
# Test Saturday is not a weekday
assert not is_weekday()

1
5


In [10]:
from typing import (
    List,
    Dict,
)

from utils import (
    get_json,
    access_nested_map,
    memoize,
)


class GithubOrgClient:
    """A Githib org client
    """
    ORG_URL = "https://api.github.com/orgs/{org}"

    def __init__(self, org_name: str) -> None:
        """Init method of GithubOrgClient"""
        self._org_name = org_name

    @memoize
    def org(self) -> Dict:
        """Memoize org"""
        return get_json(self.ORG_URL.format(org=self._org_name))

    @property
    def _public_repos_url(self) -> str:
        """Public repos URL"""
        return self.org["repos_url"]

    @memoize
    def repos_payload(self) -> Dict:
        """Memoize repos payload"""
        return get_json(self._public_repos_url)

    def public_repos(self, license: str = None) -> List[str]:
        """Public repos"""
        json_payload = self.repos_payload
        public_repos = [
            repo["name"] for repo in json_payload
            if license is None or self.has_license(repo, license)
        ]

        return public_repos

    @staticmethod
    def has_license(repo: Dict[str, Dict], license_key: str) -> bool:
        """Static: has_license"""
        assert license_key is not None, "license_key cannot be None"
        try:
            has_license = access_nested_map(
                repo, ("license", "key")) == license_key
        except KeyError:
            return False
        return has_license


In [11]:
google = GithubOrgClient("google")

In [12]:
print(google.org)

{'login': 'google', 'id': 1342004, 'node_id': 'MDEyOk9yZ2FuaXphdGlvbjEzNDIwMDQ=', 'url': 'https://api.github.com/orgs/google', 'repos_url': 'https://api.github.com/orgs/google/repos', 'events_url': 'https://api.github.com/orgs/google/events', 'hooks_url': 'https://api.github.com/orgs/google/hooks', 'issues_url': 'https://api.github.com/orgs/google/issues', 'members_url': 'https://api.github.com/orgs/google/members{/member}', 'public_members_url': 'https://api.github.com/orgs/google/public_members{/member}', 'avatar_url': 'https://avatars.githubusercontent.com/u/1342004?v=4', 'description': 'Google ❤️ Open Source', 'name': 'Google', 'company': None, 'blog': 'https://opensource.google/', 'location': None, 'email': 'opensource@google.com', 'twitter_username': 'GoogleOSS', 'is_verified': True, 'has_organization_projects': True, 'has_repository_projects': True, 'public_repos': 2514, 'public_gists': 0, 'followers': 23123, 'following': 0, 'html_url': 'https://github.com/google', 'created_at':

In [7]:
google.public_repos()

['truth',
 'ruby-openid-apps-discovery',
 'autoparse',
 'anvil-build',
 'googletv-android-samples',
 'ChannelPlate',
 'GL-Shader-Validator',
 'qpp',
 'CSP-Validator',
 'embed-dart-vm',
 'module-server',
 'cxx-std-draft',
 'filesystem-proposal',
 'libcxx',
 'tracing-framework',
 'namebench',
 'devtoolsExtended',
 'sirius',
 'testRunner',
 'crx2app',
 'episodes.dart',
 'cpp-netlib',
 'dagger',
 'ios-webkit-debug-proxy',
 'google.github.io',
 'kratu',
 'build-debian-cloud',
 'traceur-compiler',
 'firmata.py',
 'vector_math.dart']